# Plateforme Agnostique de Traitement et d'Analyse des Textes
### Carnet d'expérimentation
---

## Sujet : 

---

# Observations et environnement
---

## Environnement

In [1]:
random_state = 42

In [2]:
cd ../..

/Volumes/Geek/Work/Patat


In [3]:
import ast
import importlib
import pandas as pd
import seaborn as sns
from tqdm.notebook import tqdm
tqdm.pandas()

## Données

In [4]:
import patat.util.file

In [5]:
sitemaps_xml = patat.util.file.pickle_load('data/tmp/sitemap_xml.pickle')

## Exploration Sitemaps

In [6]:
from bs4 import BeautifulSoup

In [8]:
sitemap_data = []
for i in tqdm(range(0,len(sitemaps_xml))):
    soup = BeautifulSoup(sitemaps_xml[i]['xml'],parser='xml')
    for s_url in soup.select('url'):
        urls = s_url.select('loc')
        lastmods = s_url.select('lastmod')
        if len(urls)>0:
            url = urls[0].text
        else:
            url = ''
        if len(lastmods)>0:
            lastmod = lastmods[0].text
        sitemap_data.append({
            'url':url,
            'lastmod':lastmod,
        })


  0%|          | 0/433 [00:00<?, ?it/s]

/opt/homebrew/Caskroom/miniforge/base/envs/pt39/lib/python3.9/site-packages/bs4/builder/__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(


In [9]:
df_sitemap = pd.DataFrame(sitemap_data)
df_sitemap

,url,lastmod
0,https://bonsens.info/tout-ce-que-lon-ne-nous-d...,2020-10-03T19:13:17+00:00
1,https://bonsens.info/naissance-de-bonsens/,2020-10-04T10:13:04+00:00
2,https://bonsens.info/evenements-bonsens/,2020-10-17T16:03:37+00:00
3,https://bonsens.info/communique-de-presse-n2-2...,2020-10-31T13:36:21+00:00
4,https://bonsens.info/communique-de-presse-n1-1...,2020-10-31T13:36:54+00:00
...,...,...
906608,https://www.bvoltaire.fr/rubrique/tribune/,2023-05-08T19:08:35+00:00
906609,https://www.bvoltaire.fr/rubrique/union-europe...,2023-05-06T07:02:37+00:00
906610,https://www.bvoltaire.fr/rubrique/videos/,2023-05-05T17:56:04+00:00
906611,https://www.bvoltaire.fr/rubrique/videos-exter...,2023-02-06T20:31:41+00:00


In [10]:
import patat.imp.importer
imp = patat.imp.importer.Importer()

In [11]:
df_sitemap['site']=df_sitemap['url'].progress_apply(imp.get_site)

  0%|          | 0/906613 [00:00<?, ?it/s]

In [12]:
df_sitemap.value_counts('site')

site
www.cnews.fr                         305621
www.francesoir.fr                    154382
francais.rt.com                       79043
www.breizh-info.com                   74910
www.lelibrepenseur.org                55977
www.bvoltaire.fr                      48709
planetes360.fr                        43750
www.profession-gendarme.com           27609
www.alnas.fr                          19611
lecourrierdesstrateges.fr             16515
                                      16351
www.polemia.com                       13570
www.les-crises.fr                     12181
infodujour.fr                          8356
lesmoutonsrebelles.com                 7370
lalettrepatriote.com                   6993
lemediaen442.fr                        5251
lezarceleurs.blogspot.com              4559
www.revue-elements.com                 2721
www.la-petite-souris-normande.com      1520
extime.fr                               813
www.dreuz.info                          531
bonsens.info               

In [13]:
df_sitemap = df_sitemap[df_sitemap['url'].notna()]

In [14]:
df_sitemap['lastmod'].describe()

count                   906613
unique                  524112
top       2014-03-08T19:28:46Z
freq                    130888
Name: lastmod, dtype: object

In [15]:
df_sitemap['date']=pd.to_datetime(df_sitemap['lastmod'],format='mixed') 

In [16]:
import datetime

In [22]:
mask_2022 = (df_sitemap['lastmod']>=('2022-01-01')) & (df_sitemap['lastmod']<('2023-01-01'))

In [23]:
df_sitemap[mask_2022].value_counts('site')

site
www.francesoir.fr                    152307
www.profession-gendarme.com           27609
www.cnews.fr                          26126
planetes360.fr                        18570
francais.rt.com                        5933
www.bvoltaire.fr                       4626
www.alnas.fr                           2439
lemediaen442.fr                        2368
lezarceleurs.blogspot.com              2275
lalettrepatriote.com                   1748
lesmoutonsrebelles.com                  923
infodujour.fr                           884
www.polemia.com                         708
www.revue-elements.com                  486
www.la-petite-souris-normande.com       373
extime.fr                               186
bonsens.info                            149
www.dreuz.info                           26
Name: count, dtype: int64

In [24]:
site = 'www.dreuz.info'

In [25]:
df_sitemap[mask_2022&(df_sitemap['site']==site)]

,url,lastmod,site,date
1577,https://www.dreuz.info/whatsapp-video-2022-01-...,2022-01-05T15:21:29Z,www.dreuz.info,2022-01-05 15:21:29+00:00
1578,https://www.dreuz.info/2022/01/le-couple-presi...,2022-01-08T21:18:57Z,www.dreuz.info,2022-01-08 21:18:57+00:00
1579,https://www.dreuz.info/2022/03/la-russie-decle...,2022-02-24T17:55:07Z,www.dreuz.info,2022-02-24 17:55:07+00:00
1580,https://www.dreuz.info/2022/03/operation-de-sa...,2022-03-14T08:30:29Z,www.dreuz.info,2022-03-14 08:30:29+00:00
1581,https://www.dreuz.info/2022/04/mort-de-jeremy-...,2022-04-06T08:06:37Z,www.dreuz.info,2022-04-06 08:06:37+00:00
1582,https://www.dreuz.info/2022/04/mort-de-jeremy-...,2022-04-06T09:49:24Z,www.dreuz.info,2022-04-06 09:49:24+00:00
1583,https://www.dreuz.info/2022/04/attentat-au-coe...,2022-04-07T18:48:52Z,www.dreuz.info,2022-04-07 18:48:52+00:00
1584,https://www.dreuz.info/2022/04/attentat-au-coe...,2022-04-07T18:48:53Z,www.dreuz.info,2022-04-07 18:48:53+00:00
1585,https://www.dreuz.info/2022/04/attentat-au-coe...,2022-04-07T18:55:15Z,www.dreuz.info,2022-04-07 18:55:15+00:00
1586,https://www.dreuz.info/2022/04/attentat-au-coe...,2022-04-07T18:55:15Z,www.dreuz.info,2022-04-07 18:55:15+00:00


# Experience
---

## Etape 1

# Sauvegarde des résultats
---

# Conclusions
---

# Bricolages
---